About the datasate
1. id: unique id for a news article
2. title: the title of news article
3. auther: auther of the news article
4. text: the text of the article
5. label: a lable that marks whether the news article is real or fake



Imorting Dependencies

In [1]:
import numpy as np
import pandas as pd
import re # re means regular expression which is useful for searching text in document
from nltk.corpus import stopwords # nltk stands for natural lang toolkit,
                                  # corpus means the body of that perticular text the imp content of the text
                                  # stopwords means those words which doesn't add much value to the paragraph vertex
from nltk.stem.porter import PorterStemmer # this is called stemming it takes the word and removes the prefix and suffix of the word and returns the root word of it
from sklearn.feature_extraction.text import TfidfVectorizer # this cunction is used for converting text into feature vectors
from sklearn.model_selection import train_test_split # this function is used for to split our dataset into trainig data and test data
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-Processing

In [4]:
# loading the dataset
news_dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')

In [5]:
news_dataset.shape

(20800, 5)

In [6]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
# counting the numbers of missing values of dataset
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
# replacing the null values with empty string
news_dataset=news_dataset.fillna('')

In [9]:
# emerging the author name and news title
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [10]:
Content=news_dataset['content']

In [11]:
print(Content)

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [12]:
# separating the data and label
x = news_dataset.drop('label', axis=1) # this x variable contains the id, title, author & text colummn
y = news_dataset['label'] # this y variable contains only the lable column

In [13]:
print(x)
print(y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming

Stemming is the process of reducing a word to its root word

example- Programming, programs, programmer =*italicised text* Program

In [15]:
# Initialize the Porter Stemmer
port_stem = PorterStemmer()

def stemming(content):
    # Remove non-alphabetic characters and split the content into words
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()

    # Stem each word and remove stopwords
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if word not in stopwords.words('english')]

    # Join the stemmed words back into a single string
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content

# Applying this function to the content
news_dataset['content'] = news_dataset['content'].apply(stemming)


In [16]:
print(news_dataset['content'])

0        darrel lucu hou dem aid even see comey letter ...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exerci b...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [17]:
# separating the data and lable
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [18]:
print(X)

['darrel lucu hou dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exerci balkan'
 'david swanson keep f aliv']


In [19]:
print(Y)

[1 0 1 ... 0 1 1]


Converting the texual data to numerical data

In [20]:
vectorizer=TfidfVectorizer()
vectorizer.fit(X)
X=vectorizer.transform(X)
print(X)

  (0, 15553)	0.28485063562728646
  (0, 13348)	0.2565896679337957
  (0, 8822)	0.3635963806326075
  (0, 8546)	0.29212514087043684
  (0, 7612)	0.24785219520671603
  (0, 6930)	0.21874169089359144
  (0, 4917)	0.233316966909351
  (0, 3759)	0.2705332480845492
  (0, 3567)	0.3598939188262559
  (0, 2933)	0.2468450128533713
  (0, 2464)	0.3676519686797209
  (0, 263)	0.27010124977708766
  (1, 16656)	0.3025156488372128
  (1, 6744)	0.19152496072048605
  (1, 5440)	0.7186013955384664
  (1, 3535)	0.2653147533915268
  (1, 2790)	0.19208753385709676
  (1, 2206)	0.36915639258038363
  (1, 1877)	0.15614790568229528
  (1, 1481)	0.2957471154505952
  (2, 15478)	0.41544962664721613
  (2, 9532)	0.49351492943649944
  (2, 5903)	0.3474613386728292
  (2, 5326)	0.3866530551182615
  (2, 3072)	0.46097489583229645
  :	:
  (20797, 12998)	0.2483705036831893
  (20797, 12227)	0.27276402145717243
  (20797, 12022)	0.24790022252744132
  (20797, 10216)	0.0804189541935242
  (20797, 9500)	0.17463635692029988
  (20797, 9431)	0.29394

Splitting the dataset into training and test data

In [21]:
X_train, X_test, Y_trian, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

Model training using Logistic regression

In [22]:
Model = LogisticRegression()

In [24]:
Model.fit(X_train, Y_trian)

LogisticRegression()

Evaluation

Accuracy score
The model will ask to predict the values and the model prediction compare to the original

In [26]:
# accuracy score of training data
X_train_predict = Model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_predict, Y_trian)

In [28]:
print("Accuracy score ",training_data_accuracy)

Accuracy score  0.9866586538461538


In [29]:
# accuracy score of the test data
X_test_predict = Model.predict(X_test)
testining_data_accuracy = accuracy_score(X_test_predict, Y_test)

In [31]:
print("Testing Accuracy score",testining_data_accuracy)

Testing Accuracy score 0.9790865384615385


# Prediciton System

In [39]:
X_new = X_test[int(input())]
prediction = Model.predict(X_new)
print(prediction)
if (prediction[0]==0):
  print("the news is real")
else:
  print("the news is fake")

print("Actual value ",Y_test[0])

4
[0]
the news is real


In [40]:
index = int(input())
X_new = X_test[index]
Y_actual = Y_test[index]

prediction = Model.predict(X_new)
print(prediction)

if prediction[0] == 0:
    print("The news is real")
else:
    print("The news is fake")

print("Actual value:", Y_actual)  # Print the actual value corresponding to X_new


6
[0]
The news is real
Actual value: 0
